In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import scipy
import skimage
# import custom functions
import sys
sys.path.append("../../../../../")
import utils
import os
from tqdm import tqdm
import torch
from skimage.io import imread
from scipy.io import loadmat
from skimage.measure import regionprops
from skimage.transform import resize
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import re
import graph


In [2]:
# load data
df = pd.read_csv('../../../../../../data/codex_murine/features_and_metadataV2.csv', index_col = 0)
#df.head()

In [3]:
# save out
# feature edge
# feature label
# cell nbhd
# spatial edge

features_names = [
        'CD45',
        'Ly6C',
        'TCR',
        'Ly6G',
        'CD19',
        'CD169',
        'CD106',
        'CD3',
        'CD1632',
        'CD8a',
        'CD90',
        'F480',
        'CD11c',
        'Ter119',
        'CD11b',
        'IgD',
        'CD27',
        'CD5',
        'CD79b',
        'CD71',
        'CD31',
        'CD4',
        'IgM',
        'B220',
        'ERTR7',
        'MHCII',
        'CD35',
        'CD2135',
        'CD44',
        'nucl',
        'NKp46'
    ]
features = df[features_names].to_numpy()
features = utils.center_scale(features)


In [4]:
data_dir = '../data/'

In [5]:
## add for spatial edge
locations = df[['centroid_x', 'centroid_y']].to_numpy()
spatial_edges = graph.get_spatial_edges(
    arr=locations, n_neighbors=15, verbose=True
)
np.save(os.path.join(data_dir, f'spatial_edges_0326.npy'), np.array(spatial_edges[:2]).T)

Constructing the graph...


/home/bokai/miniconda3/envs/cellsnap/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Done!


## Create resolution test

test resolution influence

testing 0.2, 0.5, 0.7, 1.0 (set neigh size to 20)

In [7]:
res = 0.2
knn = 20

feature_edges = graph.get_feature_edges(
    arr=features, pca_components=25,
    n_neighbors=15, metric='correlation', verbose=False
)

feature_labels = graph.graph_clustering(
    df.shape[0], feature_edges, resolution=res, n_clusters=None, n_runs=1,
    resolution_tol=0.05, seed=None, verbose=False
)

np.save(os.path.join(data_dir, f'feature_labels_res{res}.npy'), feature_labels)
np.save(os.path.join(data_dir, f'feature_edges_res{res}.npy'), np.array(feature_edges[:2]).T)

print('Calculating cell neighborhood composition matrix...')
locations = df[['centroid_x', 'centroid_y']].to_numpy()
spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=knn, method='kd_tree')
cell_nbhd = utils.get_neighborhood_composition(knn_indices=spatial_knn_indices, labels=feature_labels)
np.save(os.path.join(data_dir, f'cell_nbhd_res{res}_k{knn}.npy'), cell_nbhd)

## The spatial edges were not reproduced on the spleen dataset as it would alway remain the same as long as linking NN-15 cells.
#spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=15, method='kd_tree')
#np.save(os.path.join(data_dir, f'spatial_knn_indices_k15.npy'), spatial_knn_indices) # for CNN smoothing


/home/bokai/miniconda3/envs/cellsnap/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Calculating cell neighborhood composition matrix...


In [6]:
res = 0.5
knn = 20

feature_edges = graph.get_feature_edges(
    arr=features, pca_components=25,
    n_neighbors=15, metric='correlation', verbose=False
)

feature_labels = graph.graph_clustering(
    df.shape[0], feature_edges, resolution=res, n_clusters=None, n_runs=1,
    resolution_tol=0.05, seed=None, verbose=False
)

np.save(os.path.join(data_dir, f'feature_labels_res{res}.npy'), feature_labels)
np.save(os.path.join(data_dir, f'feature_edges_res{res}.npy'), np.array(feature_edges[:2]).T)

print('Calculating cell neighborhood composition matrix...')
locations = df[['centroid_x', 'centroid_y']].to_numpy()
spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=knn, method='kd_tree')
cell_nbhd = utils.get_neighborhood_composition(knn_indices=spatial_knn_indices, labels=feature_labels)
np.save(os.path.join(data_dir, f'cell_nbhd_res{res}_k{knn}.npy'), cell_nbhd)

## save out spatial edge, always 15 cells
#spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=15, method='kd_tree')
#np.save(os.path.join(data_dir, f'spatial_knn_indices_k15.npy'), spatial_knn_indices) # for CNN smoothing


/home/bokai/miniconda3/envs/cellsnap/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Calculating cell neighborhood composition matrix...


In [7]:
len(np.unique(feature_labels))

14

In [9]:
res = 0.7
knn = 20

feature_edges = graph.get_feature_edges(
    arr=features, pca_components=25,
    n_neighbors=15, metric='correlation', verbose=False
)

feature_labels = graph.graph_clustering(
    df.shape[0], feature_edges, resolution=res, n_clusters=None, n_runs=1,
    resolution_tol=0.05, seed=None, verbose=False
)

np.save(os.path.join(data_dir, f'feature_labels_res{res}.npy'), feature_labels)
np.save(os.path.join(data_dir, f'feature_edges_res{res}.npy'), np.array(feature_edges[:2]).T)

print('Calculating cell neighborhood composition matrix...')
locations = df[['centroid_x', 'centroid_y']].to_numpy()
spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=knn, method='kd_tree')
cell_nbhd = utils.get_neighborhood_composition(knn_indices=spatial_knn_indices, labels=feature_labels)
np.save(os.path.join(data_dir, f'cell_nbhd_res{res}_k{knn}.npy'), cell_nbhd)

## save out spatial edge, always 15 cells
#spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=15, method='kd_tree')
#np.save(os.path.join(data_dir, f'spatial_knn_indices_k15.npy'), spatial_knn_indices) # for CNN smoothing


Calculating cell neighborhood composition matrix...


In [10]:
res = 1
knn = 20

feature_edges = graph.get_feature_edges(
    arr=features, pca_components=25,
    n_neighbors=15, metric='correlation', verbose=False
)

feature_labels = graph.graph_clustering(
    df.shape[0], feature_edges, resolution=res, n_clusters=None, n_runs=1,
    resolution_tol=0.05, seed=None, verbose=False
)

np.save(os.path.join(data_dir, f'feature_labels_res{res}.npy'), feature_labels)
np.save(os.path.join(data_dir, f'feature_edges_res{res}.npy'), np.array(feature_edges[:2]).T)

print('Calculating cell neighborhood composition matrix...')
locations = df[['centroid_x', 'centroid_y']].to_numpy()
spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=knn, method='kd_tree')
cell_nbhd = utils.get_neighborhood_composition(knn_indices=spatial_knn_indices, labels=feature_labels)
np.save(os.path.join(data_dir, f'cell_nbhd_res{res}_k{knn}.npy'), cell_nbhd)

## save out spatial edge, always 15 cells
#spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=15, method='kd_tree')
#np.save(os.path.join(data_dir, f'spatial_knn_indices_k15.npy'), spatial_knn_indices) # for CNN smoothing


Calculating cell neighborhood composition matrix...


## create KNN neighborhood size

In [8]:
res = 0.5
knn = 10

feature_edges = graph.get_feature_edges(
    arr=features, pca_components=25,
    n_neighbors=15, metric='correlation', verbose=False
)

feature_labels = graph.graph_clustering(
    df.shape[0], feature_edges, resolution=res, n_clusters=None, n_runs=1,
    resolution_tol=0.05, seed=None, verbose=False
)

np.save(os.path.join(data_dir, f'feature_labels_res{res}.npy'), feature_labels)
np.save(os.path.join(data_dir, f'feature_edges_res{res}.npy'), np.array(feature_edges[:2]).T)

print('Calculating cell neighborhood composition matrix...')
locations = df[['centroid_x', 'centroid_y']].to_numpy()
spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=knn, method='kd_tree')
cell_nbhd = utils.get_neighborhood_composition(knn_indices=spatial_knn_indices, labels=feature_labels)
np.save(os.path.join(data_dir, f'cell_nbhd_res{res}_k{knn}.npy'), cell_nbhd)

## save out spatial edge, always 15 cells
#spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=15, method='kd_tree')
#np.save(os.path.join(data_dir, f'spatial_knn_indices_k15.npy'), spatial_knn_indices) # for CNN smoothing


Calculating cell neighborhood composition matrix...


In [9]:
res = 0.5
knn = 20

feature_edges = graph.get_feature_edges(
    arr=features, pca_components=25,
    n_neighbors=15, metric='correlation', verbose=False
)

feature_labels = graph.graph_clustering(
    df.shape[0], feature_edges, resolution=res, n_clusters=None, n_runs=1,
    resolution_tol=0.05, seed=None, verbose=False
)

np.save(os.path.join(data_dir, f'feature_labels_res{res}.npy'), feature_labels)
np.save(os.path.join(data_dir, f'feature_edges_res{res}.npy'), np.array(feature_edges[:2]).T)

print('Calculating cell neighborhood composition matrix...')
locations = df[['centroid_x', 'centroid_y']].to_numpy()
spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=knn, method='kd_tree')
cell_nbhd = utils.get_neighborhood_composition(knn_indices=spatial_knn_indices, labels=feature_labels)
np.save(os.path.join(data_dir, f'cell_nbhd_res{res}_k{knn}.npy'), cell_nbhd)

## save out spatial edge, always 15 cells
#spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=15, method='kd_tree')
#np.save(os.path.join(data_dir, f'spatial_knn_indices_k15.npy'), spatial_knn_indices) # for CNN smoothing


Calculating cell neighborhood composition matrix...


In [10]:
res = 0.5
knn = 30

feature_edges = graph.get_feature_edges(
    arr=features, pca_components=25,
    n_neighbors=15, metric='correlation', verbose=False
)

feature_labels = graph.graph_clustering(
    df.shape[0], feature_edges, resolution=res, n_clusters=None, n_runs=1,
    resolution_tol=0.05, seed=None, verbose=False
)

np.save(os.path.join(data_dir, f'feature_labels_res{res}.npy'), feature_labels)
np.save(os.path.join(data_dir, f'feature_edges_res{res}.npy'), np.array(feature_edges[:2]).T)

print('Calculating cell neighborhood composition matrix...')
locations = df[['centroid_x', 'centroid_y']].to_numpy()
spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=knn, method='kd_tree')
cell_nbhd = utils.get_neighborhood_composition(knn_indices=spatial_knn_indices, labels=feature_labels)
np.save(os.path.join(data_dir, f'cell_nbhd_res{res}_k{knn}.npy'), cell_nbhd)

## save out spatial edge, always 15 cells
#spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=15, method='kd_tree')
#np.save(os.path.join(data_dir, f'spatial_knn_indices_k15.npy'), spatial_knn_indices) # for CNN smoothing


Calculating cell neighborhood composition matrix...


In [13]:
res = 0.5
knn = 40

feature_edges = graph.get_feature_edges(
    arr=features, pca_components=25,
    n_neighbors=15, metric='correlation', verbose=False
)

feature_labels = graph.graph_clustering(
    df.shape[0], feature_edges, resolution=res, n_clusters=None, n_runs=1,
    resolution_tol=0.05, seed=None, verbose=False
)

np.save(os.path.join(data_dir, f'feature_labels_res{res}.npy'), feature_labels)
np.save(os.path.join(data_dir, f'feature_edges_res{res}.npy'), np.array(feature_edges[:2]).T)

print('Calculating cell neighborhood composition matrix...')
locations = df[['centroid_x', 'centroid_y']].to_numpy()
spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=knn, method='kd_tree')
cell_nbhd = utils.get_neighborhood_composition(knn_indices=spatial_knn_indices, labels=feature_labels)
np.save(os.path.join(data_dir, f'cell_nbhd_res{res}_k{knn}.npy'), cell_nbhd)

## save out spatial edge, always 15 cells
#spatial_knn_indices = graph.get_spatial_knn_indices(locations=locations, n_neighbors=15, method='kd_tree')
#np.save(os.path.join(data_dir, f'spatial_knn_indices_k15.npy'), spatial_knn_indices) # for CNN smoothing


Calculating cell neighborhood composition matrix...


Below is saving out different image files for image related benchmarking

In [ ]:
def save_train_test_data(df, size, load_path, save_folder, trun_qr, shape_of_each_view=(1008, 1344),
                               channels=('CD45', 'nucl'), shape_of_views=(9, 7), pad = 1000, verbose = True):
    '''
    For all cells in the dataframe, locate it in the picture and crop the image of given size, padding 0 if needed
    Only select channels in the argument
    Save numpy array of size n_cells * size * size * n_channels
    
    
    Args:
    df is a pandas dataframe consisting of clean cells
    '''
    n_cell = df.shape[0]
    n_channel = len(channels)
    if verbose:
        print("Combining all views...", flush = True)
    ### make all view into one big view
    
    pad_image = np.zeros((shape_of_each_view[0]*shape_of_views[0], 
                          shape_of_each_view[1]*shape_of_views[1], n_channel))
    for view_j in range(shape_of_views[1]):
        for view_i in range(shape_of_views[0]):
            view = view_j * shape_of_views[0] + view_i + 1
            img = imread(
                    '{}/raw_images/focused_BALBc-1_X0{}_Y0{}.tif'.format(
                        load_path, str(view_j + 1), str(view_i + 1)
                    )
                )
            pad_image[view_i*shape_of_each_view[0]:(view_i+1)*shape_of_each_view[0],
                      view_j*shape_of_each_view[1]:(view_j+1)*shape_of_each_view[1],:] = select_useful_features(img, channels)
    
    if verbose:
        print("Done!", flush = True)
    
    zero_pad_entire_view = np.zeros((pad_image.shape[0]+2*pad, pad_image.shape[1]+2*pad, pad_image.shape[2]))
    zero_pad_entire_view[pad:pad_image.shape[0]+pad, pad:pad_image.shape[1]+pad, :] = pad_image
    truncate = np.quantile(pad_image, q = trun_qr, axis = (0, 1))
    truncate = truncate[None, None, :]
    
    print(f"the current truncation is {truncate}")
    
    zero_pad_entire_view[zero_pad_entire_view <= truncate] = 0
    zero_pad_entire_view[zero_pad_entire_view > truncate] = 1
    
    if verbose:
        print("Processing each cell...and saving!", flush = True)
    for i in tqdm(range(n_cell)):
        # process each cell
        center_x = df.iloc[i]["centroid_x"]
        center_y = df.iloc[i]["centroid_y"]
        cur_image = np.transpose(zero_pad_entire_view[(int(center_x)-size//2+pad):(int(center_x)+size//2+pad), 
                                               (int(center_y)-size//2+pad):(int(center_y)+size//2+pad), 
                                               :], (2, 0, 1)).astype(np.int8)
        assert(cur_image.shape == (2, size, size))
        if verbose:
            if i % 10000 == 1:
                plt.imshow(cur_image[0, :, :])
                plt.show()
                plt.imshow(cur_image[1, :, :])
                plt.show()
        
        np.save(file = os.path.join(save_folder, f"size{size}", "images", f"img_{i:05d}"), arr = cur_image)
        
    return
        

In [ ]:
load_path = '../../../data/codex_murine'
save_folder = os.path.join(load_path, "processed_data", "single_cell_images")

size = 1024 # or 256 save out
trun_qr = 0.9 # alpha, or 0.7, 0.8, 0.6 save out
channels = ('CD45', 'nucl')
n_channels = len(channels)
if not os.path.exists(os.path.join(save_folder, f"size{size}", "images")):
    os.makedirs(os.path.join(save_folder, f"size{size}", "images"))
save_train_test_data(df, size, load_path, save_folder, trun_qr = trun_qr, shape_of_each_view=(1008, 1344),
                                       channels=channels, shape_of_views=(9, 7), pad = 1000, verbose = True)